<a href="https://colab.research.google.com/github/peperjet/tutorial/blob/main/plotly_260205.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Plotly  Base Char(바, 산점도, 라인)
import pandas as pd
import plotly.express as px

# 데이터 불러오기 (파일 경로 확인)
df_latest = pd.read_csv('/content/sample_data/country_wise_latest.csv')

# (1) Bar Chart: 확진자 수 상위 20개국
fig_bar = px.bar(df_latest.sort_values('Confirmed', ascending=False).head(20),
                 x='Country/Region', y='Confirmed',
                 color='Confirmed', title="Top 20 Countries by Confirmed Cases")

# (2) Scatter Plot: 확진자(Confirmed) vs  사망자(Deaths) 상관관계
# 'WHO Region' 별로 색상을 나누면 분석적 입니다
fig_scatter = px.scatter(df_latest,
                         x='Confirmed',
                         y='Deaths',
                         size='Confirmed',
                         color='WHO Region',
                         hover_name='Country/Region',
                         log_x=True,
                         log_y=True,
                         title="Confirmed Cases vs Deaths (Log Scale by Region)")


# (3) Line chart: 이 데이터는 시계열이 아니지만
# 데이터의 '경향성'을 보기 위해 상위 20개국 라인으로 연결해볼 수 있습니다.
fig_line = px.line(df_latest.sort_values('Confirmed', ascending=False).head(20),
                   x='Country/Region',
                   y='Confirmed',
                   title="Trend of Top 20 Confirmed Countries")



fig_bar.show()
fig_scatter.show()
fig_line.show()

In [ ]:
# 미션 : Scientific Chart (Correlation Heatmap)

import plotly.graph_objects as go

# 1. 상관관계를 계산할 칼럼들만 선택 (대소문자 주의)
cols_to_use = ['Confirmed', 'Deaths', 'Recovered', 'Active', 'New cases']
corr_matrix = df_latest[cols_to_use].corr()

# 2. 히트맵 생성(공식문서의 go.Heatmap 인용)
fig_heat = go.Figure(data=go.Heatmap(
    z=corr_matrix.values,  # 상관계수 값
    x=corr_matrix.columns,  # x축 레이블
    y=corr_matrix.columns,  # y 축 레이블
    colorscale='Viridis',  # 공식 문서 추천 컬러 (데이터 밀도 표현에 최적)
    texttemplate="%{z:.2f}",  # 격자 안에 숫자 표시 (소수점 2자리)
    hoverongaps=False
))

fig_heat.update_layout(
    title='Scientific Chart: COVID-19 Metrics Correlation',
    width=600, height=600 # 정사각형 모양이 보기 좋습니다
)

fig_heat.show()

In [ ]:
# 미션 : 더블슬라이더 (Range Slider) 구현 - 특정 확진자 수 구간에 있는 국가들만 돋보기 확대해서 볼 수 있음
import plotly.graph_objects as go

# 1. 데이터 전처리 : 정렬 (확진자 순으로 세우기)
df_sorted = df_latest.sort_values('Confirmed')

fig_slider = go.Figure()

# 2. 국가별 확진자 수 Scatter (Line + Markers)
fig_slider.add_trace(go.Scatter(
    x=df_sorted['Country/Region'],
    y=df_sorted['Confirmed'],
    mode='lines+markers',
    marker=dict(color='skyblue', size=8), # 'market' -> 'marker'로 수정
    line=dict(color='blue', width=1),
    name='Confirmed Cases'
))

# 3. 더블 슬라이더 설정(Range Slider)
fig_slider.update_layout(
    title='Interactive Range Slider: Search Countries by Case Load',
    xaxis=dict(
        rangeslider=dict(visible=True), # 하단 슬라이더 활성화
        type='category'
    ),
    yaxis=dict(title="Confirmed Cases"),
    template="plotly_white",
    height=600
)

fig_slider.show()

In [ ]:
with open('my_covid_dashboard.html', 'a') as f:
    # 1번: 통계 차트(박스플롯) 추가
    f.write(fig_bar.to_html(full_html=False, include_plotlyjs='cdn'))

    # 2번: 드롭다운/버튼 차트 추가
    f.write(fig_scatter.to_html(full_html=False, include_plotlyjs='notjs'))

    # 3번: 히트맵 추가
    f.write(fig_heat.to_html(full_html=False, include_plotlyjs='notjs'))

    # 4번: 슬라이더 차트 추가
    f.write(fig_slider.to_html(full_html=False, include_plotlyjs='notjs'))

print("대시보드가 'my_covid_dashboard.html'로 저장되었습니다!")

대시보드가 'my_covid_dashboard.html'로 저장되었습니다!


In [24]:
# Plotly Scientific Chart (1. 상관관계 히트맵)
# Heatmaps 섹션에서는 z, x, y 인자를 사용해 데이터 밀도를 표현

import plotly.graph_objects as go
# 확진자, 사망자, 회복자 등 수치형 데이터간 상관관계 계산
numeric_df = df_latest.select_dtypes(include=['number']).corr()

fig = go.Figure(data=go.Heatmap(
    z=numeric_df.values,
    x=numeric_df.columns,
    y=numeric_df.columns,
    colorscale='Viridis' # 공식 문서 추천 컬러 (오타 수정)
))

fig.update_layout(title='Scientific Chart: Feature Correlation Heatmap')
fig.show()


In [39]:
# Plotly Interactive Slider (2. 시간 흐름 애니메이션)
import plotly.express as px
import pandas as pd


# 데이터 로드
df_full = pd.read_csv('/content/sample_data/full_grouped.csv')
df_full['Date'] = pd.to_datetime(df_full['Date'])

# 데이터 전처리 ( 점이 너무 많다면 상위 국가만 뽑거나 날짜를 정렬해야 함)
# df_full['Date'] = pd.to_datetime(df_full['Date'])
# df_full = df_full.sort_values('Date')

# 2. [핵심] 시작 날짜 필터링 (로딩 공백 제거)
# 4월 1일부터 시작하도록 설정
start_date = '2020-04-01'
df_filtered = df_full[df_full['Date'] >= start_date].sort_values('Date')

# 3. 날짜순으로 정렬 (애니메이션이 순서대로 흐르게 함)
df_filtered = df_filtered.sort_values('Date')

# 4. 날짜 컬럼을 문자열로 변환 (Plotly 슬라이더가 날짜 객체보다 문자열일 때 필터링을 더 잘 인식함)
df_filtered['Date_str'] = df_filtered['Date'].dt.strftime('%Y-%m-%d')


# 애니메이션 차트 생성 (반드시 df_filtered만 사용!)
fig5 = px.scatter(df_filtered,
                x="Confirmed",
                y="Deaths",
                animation_frame="Date_str", # df_filtered의 'Date_str' 컬럼 사용
                animation_group="Country/Region",
                size="Confirmed",
                color="WHO Region",
                hover_name="Country/Region",
                log_x=True,
                log_y=True, # 사망자 수도 로그 스케일 추천
                size_max=60, # 중요 : 축 범위를 데이터 전체의 최솟값~최댓값으로 고정해야 점들이 안 튭니다!
                range_x=[10, df_full['Confirmed'].max()*1.1],
                range_y=[1, df_full['Deaths'].max()*1.1])

fig5.update_layout(title=f'COVID-19 Global Progression (Started from {start_date})')

fig5.show()

In [40]:
# 통합 대시보드 저장용 코드
dashboard_name = 'COVID19_Mission_Complete.html'

with open(dashboard_name, 'w', encoding='utf-8') as f:
    # 1. HTML 기본 구조 및 스타일 설정
    f.write("<html><head><title>COVID-19 Analysis Dashboard</title>")
    f.write("<style>body { font-family: sans-serif; margin: 40px; background-color: #f4f7f6; } ")
    f.write("h1, h2 { color: #2c3e50; } .chart-container { margin-bottom: 50px; background: white; padding: 20px; border-radius: 10px; box-shadow: 0 4px 6px rgba(0,0,0,0.1); }</style>")
    f.write("</head><body>")

    f.write("<h1>🚀 COVID-19 Interactive Data Dashboard</h1>")
    f.write("<p>Plotly 공식 문서와 Kaggle 데이터를 활용한 시각화 미션 최종 결과물입니다.</p>")
    f.write("<hr>")

    # 2. 각 차트 섹션 추가 (차트 변수명을 본인이 설정한 이름과 확인하세요!)

    # 섹션 1: Basic Chart (Bar)
    f.write("<div class='chart-container'><h2>1. Top 20 Countries (Bar Chart)</h2>")
    f.write(fig_bar.to_html(full_html=False, include_plotlyjs='cdn'))
    f.write("</div>")

    # 섹션 2: Scientific Chart (Heatmap)
    f.write("<div class='chart-container'><h2>2. Metrics Correlation (Heatmap)</h2>")
    f.write(fig_heat.to_html(full_html=False, include_plotlyjs='notjs'))
    f.write("</div>")

    # 섹션 3: Interactive Range Slider (Double)
    f.write("<div class='chart-container'><h2>3. Country Range Explorer (Double Slider)</h2>")
    f.write(fig_slider.to_html(full_html=False, include_plotlyjs='notjs'))
    f.write("</div>")

    # 섹션 4: Animation Slider (Starting from March)
    f.write("<div class='chart-container'><h2>4. Global Progression Animation (Single Slider)</h2>")
    f.write(fig5.to_html(full_html=False, include_plotlyjs='notjs'))
    f.write("</div>")

    f.write("</body></html>")

print(f"🎉 성공! '{dashboard_name}' 파일이 생성되었습니다. 왼쪽 폴더 아이콘을 눌러 다운로드하세요!")

🎉 성공! 'COVID19_Mission_Complete.html' 파일이 생성되었습니다. 왼쪽 폴더 아이콘을 눌러 다운로드하세요!
